# Working

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("src/")

In [6]:

from src.data import get_track , getBatch
from src.model import PCT_Transformer
import glob
import numpy as np
import tensorflow as tf

In [7]:
aPaths = glob.glob("data/*AllPSA.npy")
hpaths = glob.glob("data/*.hits.npy")
ap = np.random.choice(aPaths)
hp = np.random.choice(hpaths)
trk =  get_track(apth=ap,hpth=hp)
batch = getBatch(trk,batch_size=32)
X = batch[:,:,2:5]
Y = batch[:,:,5:]
model = PCT_Transformer()

In [8]:
model.compile(optimizer = tf.keras.optimizers.Adam(),loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False))

In [9]:
model.fit(X,Y)

Epoch:  0
Loss:  23.721388


(<tf.Tensor: shape=(), dtype=float32, numpy=23.721388>,
 <tf.Tensor: shape=(32, 3), dtype=float32, numpy=
 array([[0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.03980862, 0.49929404, 0.4608972 ],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.28908902, 0.41995385, 0.29095706],
        [0.03980862, 0.49929404, 0.4608972 ],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 , 0.2942406 , 0.48122323],
        [0.2245361 ,

# Experiment

In [43]:
predictions, loss = model(X,Y)

In [44]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=249.33801>

In [9]:
import numpy as np
import tensorflow as tf
from scipy.spatial import distance_matrix

N=200

xINP = np.random.uniform(-1,1,(N,2))
permutation = np.random.permutation(range(N))
yINP = np.array(list(map(lambda i: xINP[i], permutation)))
Noise = 0.01
Noise = 0.1
yINP += np.random.normal(0,Noise,(N,2))


def bipartete_matching(xINP:np.ndarray,yINP:np.ndarray)-> (np.ndarray,np.ndarray):
    cost_matrix = distance_matrix(xINP,yINP)

    queue = []
    for i in range(cost_matrix.shape[0]):
        for j in range(cost_matrix.shape[1]):
            queue.append([i,j,cost_matrix[i,j]])
    queue = sorted(queue, key=lambda x: x[2])

    row_to_col_match_vec = np.full(N,-1)
    col_to_row_match_vec = np.full(N,-1)
    for row in queue:
        if (row_to_col_match_vec[row[0]] == -1) & (col_to_row_match_vec[row[1]] == -1):
            row_to_col_match_vec[row[0]] = row[1]
            col_to_row_match_vec[row[1]] = row[0]
    return col_to_row_match_vec

In [15]:
class BipartateMatching(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    def __call__(self,xINP:tf.Tensor,yINP:tf.Tensor)->tf.Tensor:
        cost_matrix = distance_matrix(xINP,yINP)

        queue = []
        for i in range(cost_matrix.shape[0]):
            for j in range(cost_matrix.shape[1]):
                queue.append([i,j,cost_matrix[i,j]])
        queue = sorted(queue, key=lambda x: x[2])

        row_to_col_match_vec = np.full(N,-1)
        col_to_row_match_vec = np.full(N,-1)
        for row in queue:
            if (row_to_col_match_vec[row[0]] == -1) & (col_to_row_match_vec[row[1]] == -1):
                row_to_col_match_vec[row[0]] = row[1]
                col_to_row_match_vec[row[1]] = row[0]
        return tf.convert_to_tensor(col_to_row_match_vec)

In [16]:
lx = BipartateMatching()
lx(tf.convert_to_tensor(xINP),tf.convert_to_tensor(yINP))

<tf.Tensor: shape=(200,), dtype=int32, numpy=
array([187, 138,  38, 147,  35, 112, 104,  13,   9,  77,  81,  57, 105,
        32,  11,  91, 134, 127, 137, 120, 168,  53,  41, 152,  93,  16,
       158, 119,  21,  29,  62, 198,  14,  96,  66, 181, 135, 111, 157,
       150, 162,   1, 117,  20, 194,  22,  26, 180, 188,  76, 179, 118,
        51, 167, 169, 141,  27,  59,  86, 103,  80,  70, 190,  52,  58,
       172,  25,  94, 186,  23,  36, 156,   3,  69, 151, 114,  61, 166,
        65, 183,  46, 144, 116,  89,  50, 160,  79, 173, 109, 123, 184,
       199, 189,  48,  78,  40,  99,  17,  63,  64,  45, 107, 142,  75,
        68,   0,  83,   7, 122,  10,  42, 170,  43, 108,  24, 110, 129,
        98,  28,  33, 131, 174, 100, 153, 161, 191,   4, 185,  71, 133,
        47, 126, 165,  82, 159,  67,   5, 182, 176,  19,   6, 149,  54,
       125,  73,  39, 171, 102, 196,  31, 163, 132, 101, 121, 106,  60,
       143,  74, 148, 192, 177,  95, 145,  97,  92,  37, 154,  85,  90,
       124, 175, 1